In [26]:
#import librariesImport
# import gedlibpy
import networkx as nx
import numpy as np
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
from torch_geometric.utils import from_networkx
import functools
from tqdm import tqdm
import random
import time
import os
import torch
from multiprocessing import Pool
from matplotlib import pyplot as plt
from torch_geometric.datasets import Planetoid
import signal


In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [4]:
dest_folder = "./data/"

# Filter graphs with more than max_num_nodes nodes
max_num_nodes = 20
NUM_WORKER = 16

In [61]:
# All Methods for filtering graphs

class TimeoutError(Exception):
    pass

def handler(signum, frame):
    raise TimeoutError()

def set_x(data_obj):
    data_obj.x = torch.ones(data_obj.num_nodes, 1)
    return data_obj

def node_match(n1, n2):
    return np.argmax(n1['x']) == np.argmax(n2['x'])


def get_unique_graphs_using_nx(nx_graphs, timeout_duration = 15):
    # Filter Isomorphic graphs using nx.is_isomorphic
    unique_graphs = [nx_graphs[0]]
    
    signal.signal(signal.SIGALRM, handler) 


    print("Filtering isomorphic graphs...")
    for i in tqdm(range(1, len(nx_graphs))):
        unique = True
        for j in range(len(unique_graphs)):
            g1 = nx_graphs[i]
            g2 = unique_graphs[j]
            if g1.number_of_nodes() > g2.number_of_nodes():
                g1, g2 = g2, g1

            signal.alarm(timeout_duration)
            try:
                if nx.is_isomorphic(g1, g2, node_match=node_match):
                    #print("Graph ", i, " is isomorphic to graph ", j)
                    unique = False
                    break
            except TimeoutError:
                print(f"Timeout for graph {i}")
                unique = False
                break
            finally:
                signal.alarm(0)
        if unique:
            #print("Including graph ", i, " in unique_graphs")
            unique_graphs.append(nx_graphs[i])
    print(f"Total number of non-isomorphic graphs: {len(unique_graphs)}")
    return unique_graphs


def split_and_dump_graphs(nx_graphs, data_path = None, DATASET = ""):
    print("Splitting and dumping graphs...")
    if data_path == None:
        print("ERROR: Data path not provided")
        return
    
    random.seed(0)
    random.shuffle(nx_graphs)    # Shuffle the list of graphs
    train_size = int(0.6*len(nx_graphs))
    val_size = int(0.2*len(nx_graphs))
    test_size = len(nx_graphs) - train_size - val_size

    train_graphs = nx_graphs[:train_size]
    val_graphs = nx_graphs[train_size:train_size+val_size]
    test_graphs = nx_graphs[train_size+val_size:]
    print(f"Train: {len(train_graphs)}, Val: {len(val_graphs)}, Test: {len(test_graphs)}")


    # Dump Non-isomorphic Graphs 
    train_data = []
    for graph in train_graphs:
        data = from_networkx(graph)
        train_data.append(data)
    torch.save(train_data, data_path + "train.pt")
    print(f"saved {DATASET} into {data_path}train.pt")

    val_data = []
    for graph in val_graphs:
        data = from_networkx(graph)
        val_data.append(data)
    torch.save(val_data, data_path + "val.pt")
    print(f"saved {DATASET} into {data_path}val.pt")

    test_data = []
    for graph in test_graphs:
        data = from_networkx(graph)
        test_data.append(data)
    torch.save(test_data, data_path + "test.pt")
    print(f"saved {DATASET} into {data_path}test.pt")


def check_leakage(data_path, node_match_fun = None, only_leakage = False):
    print("Checking for leakage...")
    print("loading saved splits from ", data_path)
    train_data = torch.load(data_path + "/train.pt")
    val_data = torch.load(data_path + "/val.pt")
    test_data = torch.load(data_path + "/test.pt")

    train_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), train_data))
    val_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), val_data))
    test_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), test_data))
    
    print("Number of graphs in train: ", len(train_nx), " Val: ", len(val_nx), " Test: ", len(test_nx))

    if not only_leakage:
        cnt = 0
        for i in tqdm(range(len(train_nx))):
            for j in range(len(train_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(train_nx[i], train_nx[j], node_match=node_match):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicate inside train: ", cnt)
        
        cnt = 0
        for i in tqdm(range(len(val_nx))):
            for j in range(len(val_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(val_nx[i], val_nx[j], node_match=node_match):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicates inside val:", cnt)
        
        cnt = 0
        for i in tqdm(range(len(test_nx))):
            for j in range(len(test_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(test_nx[i], test_nx[j], node_match=node_match):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicates inside test:", cnt)


    cnt = 0
    for val_g in tqdm(val_nx):
        for train_g in train_nx:
            if nx.is_isomorphic(train_g, val_g, node_match=node_match):
                #print("Isomorphic graph found between train and val")
                cnt += 1
                break
    print("Number of leaks in Val from train: ",cnt)

    cnt = 0
    for test_g in tqdm(test_nx):
        for train_g in train_nx:
            if nx.is_isomorphic(train_g, test_g, node_match=node_match):
                #print("Isomorphic graph found between train and val")
                cnt += 1
                break
    print("Number of leaks in test from train: ",cnt)

def generate_uniqe_graphs(graphs, DATASET, node_labeled = False, only_leakage = False):
    """
        graphs: list of torch_geometric.data.Data objects
            data[i].x: Node features of graph i
        DATASET: Name of the dataset

    """
    data_path = './data/'+ DATASET + '/'
    if not os.path.isdir(data_path):
        os.mkdir(data_path)

    print("Total number of graphs: ", len(graphs))
    print("Selecting Small Graphs...")
    graphs = list(filter(lambda x: x.num_nodes <= max_num_nodes, graphs))
    print(f"Graphs with at most {max_num_nodes} nodes: {len(graphs)}")

    if not node_labeled:
        graphs = list(map(set_x, graphs))

    nx_graphs = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), graphs))

    unique_nx_graphs = get_unique_graphs_using_nx(nx_graphs)
    sizes = list(map(lambda x: x.number_of_nodes(), unique_nx_graphs))
    print("Maximum number of nodes in the dataset: ", max(sizes))
    split_and_dump_graphs(unique_nx_graphs, data_path, DATASET)
    check_leakage(data_path, only_leakage=only_leakage)

# AIDS

In [62]:
DATASET = 'AIDS'
data = TUDataset(root="/tmp/", name=DATASET)
node_labeled = True

In [63]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  2000
Selecting Small Graphs...
Graphs with at most 20 nodes: 1666
Filtering isomorphic graphs...


100%|██████████| 1665/1665 [00:18<00:00, 88.31it/s] 


Total number of non-isomorphic graphs: 1588
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 952, Val: 317, Test: 319
saved AIDS into ./data/AIDS/train.pt
saved AIDS into ./data/AIDS/val.pt
saved AIDS into ./data/AIDS/test.pt
Checking for leakage...
loading saved splits from  ./data/AIDS/
Number of graphs in train:  952  Val:  317  Test:  319


100%|██████████| 952/952 [00:11<00:00, 79.77it/s]


Number of duplicate inside train:  0


...

Number of duplicates inside test: 0


100%|██████████| 317/317 [00:03<00:00, 80.35it/s]


Number of leaks in Val from train:  0


100%|██████████| 319/319 [00:04<00:00, 77.49it/s]

Number of leaks in test from train:  0


In [64]:
dummy = torch.load('./data/AIDS/train.pt')

In [67]:
dummy[2]

Data(x=[11, 38], edge_index=[2, 24])

# IMDB-BINARY

In [17]:
DATASET = 'IMDB-BINARY'
data = TUDataset(root="/tmp/", name=DATASET)


In [16]:
data[0]

Data(edge_index=[2, 146], y=[1], num_nodes=20)

In [18]:
node_labeled = False

In [19]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  1000
Selecting Small Graphs...
Graphs with at most 20 nodes: 696
Filtering isomorphic graphs...


100%|██████████| 695/695 [00:01<00:00, 640.87it/s] 


Total number of non-isomorphic graphs: 280
Splitting and dumping graphs...
Train: 168, Val: 56, Test: 56
saved IMDB-BINARY into ./data/IMDB-BINARY/train.pt
saved IMDB-BINARY into ./data/IMDB-BINARY/val.pt
saved IMDB-BINARY into ./data/IMDB-BINARY/test.pt
Checking for leakage...
loading saved splits from  ./data/IMDB-BINARY/
Number of graphs in train:  168  Val:  56  Test:  56


100%|██████████| 168/168 [00:00<00:00, 648.66it/s]


Number of duplicate inside train:  0


...

Number of duplicates inside test: 0


100%|██████████| 56/56 [00:00<00:00, 601.28it/s]


Number of leaks in Val from train:  0


100%|██████████| 56/56 [00:00<00:00, 644.15it/s]

Number of leaks in test from train:  0


# PROTEINS

In [31]:
DATASET = 'PROTEINS'
data = TUDataset(root="/tmp/", name=DATASET)


In [32]:
data[0]

Data(edge_index=[2, 162], x=[42, 3], y=[1])

In [33]:
node_labeled = False

In [34]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  1113
Selecting Small Graphs...
Graphs with at most 20 nodes: 412
Filtering isomorphic graphs...


100%|██████████| 411/411 [00:00<00:00, 655.90it/s] 


Total number of non-isomorphic graphs: 297
Splitting and dumping graphs...
Train: 178, Val: 59, Test: 60
saved PROTEINS into ./data/PROTEINS/train.pt
saved PROTEINS into ./data/PROTEINS/val.pt
saved PROTEINS into ./data/PROTEINS/test.pt
Checking for leakage...
loading saved splits from  ./data/PROTEINS/
Number of graphs in train:  178  Val:  59  Test:  60


100%|██████████| 178/178 [00:00<00:00, 576.49it/s]


Number of duplicate inside train:  0


...

Number of duplicates inside test: 0


100%|██████████| 59/59 [00:00<00:00, 533.76it/s]


Number of leaks in Val from train:  0


100%|██████████| 60/60 [00:00<00:00, 566.46it/s]

Number of leaks in test from train:  0


# LINUX

In [35]:
from  torch_geometric.datasets.ged_dataset import GEDDataset

In [36]:
DATASET = 'LINUX'

train_data = GEDDataset('/tmp/', name = "LINUX")
test_data = GEDDataset('/tmp/', name = "LINUX", train = False)
data = train_data + test_data

In [37]:
data[0]

Data(edge_index=[2, 18], i=[1], num_nodes=8)

In [38]:
node_labeled = False

In [39]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  1000
Selecting Small Graphs...
Graphs with at most 20 nodes: 1000
Filtering isomorphic graphs...


100%|██████████| 999/999 [00:00<00:00, 1486.88it/s]


Total number of non-isomorphic graphs: 89
Splitting and dumping graphs...
Train: 53, Val: 17, Test: 19
saved LINUX into ./data/LINUX/train.pt
saved LINUX into ./data/LINUX/val.pt
saved LINUX into ./data/LINUX/test.pt
Checking for leakage...
loading saved splits from  ./data/LINUX/
Number of graphs in train:  53  Val:  17  Test:  19


100%|██████████| 53/53 [00:00<00:00, 1254.40it/s]


Number of duplicate inside train:  0


...

Number of duplicates inside test: 0


100%|██████████| 17/17 [00:00<00:00, 1262.63it/s]


Number of leaks in Val from train:  0


100%|██████████| 19/19 [00:00<00:00, 1217.62it/s]

Number of leaks in test from train:  0


# OGBG

In [40]:
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader


## ogbg-molhiv

In [43]:
DATASET = 'ogbg-molhiv'
data = PygGraphPropPredDataset(name = DATASET) 

In [44]:
data[0]

Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)

In [7]:
node_labeled = True

452741

In [45]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  41127
Selecting Small Graphs...
Graphs with at most 20 nodes: 14923
Filtering isomorphic graphs...


 59%|█████▉    | 8863/14922 [05:50<03:59, 25.26it/s] 


KeyboardInterrupt: 

## ogbg-code2

In [46]:
DATASET = 'ogbg-code2'
data = PygGraphPropPredDataset(name = DATASET) 

In [47]:
data[0]

Data(edge_index=[2, 243], x=[244, 2], node_is_attributed=[244, 1], node_dfs_order=[244, 1], node_depth=[244, 1], y=[1], num_nodes=244)

In [48]:
# For OGBA-CODE2 Label adjustment

node_labeled = True

valid_graphs = []
for g in tqdm(data):
    if (g.num_nodes <= 20):
        labels = g.x[:, 0]
        g.x = torch.nn.functional.one_hot(labels, 97).float()
        rev1 = torch.stack((g.edge_index[1], g.edge_index[0]), 0)
        g.edge_index = torch.cat((g.edge_index, rev1), -1)
        valid_graphs.append(g)
data = valid_graphs
print("Total number valid graphs in the dataset: ", len(data))

100%|██████████| 452741/452741 [01:12<00:00, 6253.68it/s]


Total number valid graphs in the dataset:  404


In [53]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  404
Selecting Small Graphs...
Graphs with at most 20 nodes: 404
Filtering isomorphic graphs...


 31%|███▏      | 126/403 [01:05<01:33,  2.97it/s]

Timeout for graph 101


 36%|███▋      | 147/403 [01:20<02:21,  1.81it/s]

Timeout for graph 147


...

100%|██████████| 403/403 [03:57<00:00,  1.70it/s]

Timeout for graph 376
Total number of non-isomorphic graphs: 181
Splitting and dumping graphs...
Train: 108, Val: 36, Test: 37


saved ogbg-code2 into ./data/ogbg-code2/train.pt
saved ogbg-code2 into ./data/ogbg-code2/val.pt
saved ogbg-code2 into ./data/ogbg-code2/test.pt
Checking for leakage...
loading saved splits from  ./data/ogbg-code2/
Number of graphs in train:  108  Val:  36  Test:  37


 35%|███▌      | 38/108 [04:23<08:06,  6.94s/it]


KeyboardInterrupt: 

In [57]:
check_leakage('./data/ogbg-code2/', only_leakage=True)

Checking for leakage...
loading saved splits from  ./data/ogbg-code2/
Number of graphs in train:  108  Val:  36  Test:  37


100%|██████████| 36/36 [00:25<00:00,  1.39it/s]


Number of leaks in Val from train:  0


100%|██████████| 37/37 [00:08<00:00,  4.53it/s]

Number of leaks in test from train:  0


## ogbg-molpcba

In [ ]:
DATASET = 'ogbg-molpcba'
data = PygGraphPropPredDataset(name = DATASET) 